In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.io as pio
pio.renderers.default= 'notebook'

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer


In [3]:
url='../../Datasets/imdb_top_1000.csv'
df= pd.read_csv(url)

In [4]:
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [5]:
print(df.columns.tolist())

['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate', 'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director', 'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross']


In [6]:
cols_to_drop = ['Poster_Link','Runtime','Gross']
df.drop(cols_to_drop, axis=1, inplace=True)

In [43]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [44]:
df['text']= df['Series_Title']+' '+ df['Genre']+' '+ df['Overview']+ ' ' +df['Director']+' '+df['Star1']+' '+df['Star2']+' '+df['Star3']+' '+df['Star4']

In [45]:
vec= TfidfVectorizer(stop_words='english', ngram_range=(1,3))
tfidfMat= vec.fit_transform(df['text']).toarray()
print(tfidfMat.shape)

(1000, 58853)


In [47]:
sim=cosine_similarity(tfidfMat, tfidfMat).round(2)

In [48]:
def getIndexFromName(name):
    return df[df['Series_Title']==name].index[0]

In [49]:
idx= getIndexFromName("The Godfather")
print(df.iloc[idx])

Series_Title                                         The Godfather
Released_Year                                                 1972
Certificate                                                      A
Genre                                                 Crime, Drama
IMDB_Rating                                                    9.2
Overview         An organized crime dynasty's aging patriarch t...
Meta_score                                                   100.0
Director                                      Francis Ford Coppola
Star1                                                Marlon Brando
Star2                                                    Al Pacino
Star3                                                   James Caan
Star4                                                 Diane Keaton
No_of_Votes                                                1620367
text             The Godfather Crime, Drama An organized crime ...
Name: 1, dtype: object


In [57]:
def recommendMovie(movieName, k=5):
    idx= getIndexFromName(movieName)
    simScore= list(enumerate(sim [idx]))

    #sort
    simScore.sort(key=lambda x:x[1], reverse= True)

    #get movie indices
    movieIndex= [i[0] for i in simScore]

    #return top k movies

    return df.iloc[movieIndex]['Series_Title'].head(k)

In [59]:
mov= recommendMovie('The Dark Knight', 15)

In [60]:
type(mov)

pandas.core.series.Series

In [52]:
import gradio as gr

In [53]:
interface= gr.Interface(fn= recommendMovie , inputs='text', outputs='text')

In [54]:
interface.launch()

Running on local URL:  http://127.0.0.1:7861/

To create a public link, set `share=True` in `launch()`.


(<gradio.routes.App at 0x210e8edd9a0>, 'http://127.0.0.1:7861/', None)